In [3]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [4]:
model = gp.Model("VRPLTT")

Restricted license - for non-production use only - expires 2023-10-25


In [5]:
Q_min = 140
Q_max = 300
n_levels = 4
load_levels = np.array([(0.5+i)*(Q_max-Q_min)/n_levels for i in range(n_levels)])
upper = load_levels+(Q_max-Q_min)/(n_levels*2)
lower = load_levels-(Q_max-Q_min)/(n_levels*2)

In [6]:
load_levels

array([ 20.,  60., 100., 140.])

In [7]:
upper

array([ 40.,  80., 120., 160.])

In [8]:

def vel(m, h, P):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eff = 0.95
    
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))/eff
    c3 = c_Fd/eff
    
    coefs = [c3, 0, c1, -P]
    #print(f"{np.real(np.roots(coefs))=}")
    v = np.max(np.real(3.6*np.roots(coefs)))
    return min(v,30)
    #return 3.6*np.roots(coefs)
    

def time_matrix(data_matrix, load_levels,P):
    N = len(data_matrix.index)
    elevation = data_matrix.elevation.to_numpy()
    d_ij_mat = data_matrix.iloc[:, 8:].to_numpy()
    t_ij_mat = np.zeros((N, N, n_levels))
    for i in range(N):
        for j in range(N):
            if i!=j:
                d_ij = d_ij_mat[i, j]
                h = (elevation[j] - elevation[i])/d_ij
                for l in range(n_levels):
                    m = load_levels[l]
                    t_ij_mat[i, j, l] =60*d_ij/vel(m, h, P)
    return np.round(t_ij_mat,4)

The forces acting on the bicyle are: the aerodynamic drag resistance $F_{D}$, the rolling resistance $F_{R}$ and the component of the gravity force along the direction of motion $F_{G}$:

* $ F_{D} = \frac{\rho C_{D}Av^{2}}{2}$
* $ F_{R} = C_{R}mg cos(arctan(h))$
* $ F_{G} = mg sin(arctan(h))$

We have to distinguish two cases: uphill and downhill.
In uphill, the component of the gravity force along the direction of motion is a resistance force, while in downhill it helps the motion of the bicyle.
so we have, at equilibrium:

* $\mathrm{ \eta P_{traction} = (F_{D} + (F_{R} + F_{G})v }$

where $\eta$ is the efficiency of the mechanic transmission, which is assumed to be 0.95.

If we use a negative h in the formuals above when traversing an edge with negative slope, since $cos(\alpha) = cos(-\alpha)$ and $sin(\alpha) = -sin(-\alpha)$, we have just one expression for the equation to solve.

Moreover, we assume that the bicyle speed is limited in downhill for safety reasons. Within cities, a typical speed limit for for motor vehicles is 50km/h. Considering also that the regulation for pedelec bicyle allows a maximum speed of 25km/h in electric assisted mode, we set a maximum speed in downhill of 30km/h.
The delta energy can be used for recharging the battery for example, similarly to other solutions in the automaotive industry.
The maximum speed limit is imposed on uphill as well.
The strategy for solving the equation is:
* solve the equation for the two cases: (1) $P=P_{cyclist}$, and (2) $P=P_{cyclist}+P_{electric}$ 
* If the cyclist can reach a speed that is higher than the speed allowed in electric mode, use that speed, subject to the limit of maximum speed allowed,
* otherwise use the minimum value among 25 km/h and the solution of problem (2)

In practice we have $P_{electric}$ = 250W, while for $\overline {P_{cyclist}}$ we assume to have few different types of cyclist.

In [9]:
def vel1(m, h, P):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eta = 0.95
    
    P_electric = 250 #Watt
    
    c0 = P*eta
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))
    c3 = c_Fd
    
    v_max = 30 #km/h
   ### closed form solution of 3-rd degree polynomial

    # p = c1/c3
    # q = -c0/c3
    # print(np.roots([1, 0, p, q]))
    
    ###
    coefs = [c3, 0, c1, -eta*P]
    coefs_electric = [c3, 0, c1, -eta*(P+P_electric)]        
    v = np.max(np.real(3.6*np.roots(coefs)))
    v_electric = np.max(np.real(3.6*np.roots(coefs_electric)))
    if h > 0:
        if v > 25:
            print('up-cyclist only')
            return min(v, v_max)
        else:
            print('up-electric')
            return min(v_electric, 25)
    else:
        if v > 25:
            print('down-cyclist only')
            return min(v, v_max)
        else:
            print('down-electric')
            return min(v_electric, 25)

In [10]:
print('base model v=', vel(150, 0.05, 150+250))
print('new model v=', vel1(150, 0.05, 150))
print('\n')
print('base model v=', vel(150, -0.05, 150+250))
print('new model v=', vel1(150, -0.05, 150))
print('\n')
print('base model v=', vel(150, -0.01, 150+250))
print('new model v=', vel1(150, -0.01, 150))
print('\n')
print('base model v=', vel(150, -0.01, 300+250))
print('new model v=', vel1(150, -0.01, 300))
print('\n')
print('base model v=', vel(150, 0.01, 450+250))
print('new model v=', vel1(150, 0.01, 450))

base model v= 14.097188347234017
up-electric
new model v= 14.097188347234017


base model v= 30
down-cyclist only
new model v= 30


base model v= 30
down-electric
new model v= 25


base model v= 30
down-cyclist only
new model v= 28.44330666117872


base model v= 30
up-cyclist only
new model v= 25.918030223661823


In [11]:
df = pd.read_csv("./instances/small/Fukuoka_01.csv")
P = 500 # Watt

t=time_matrix(df,load_levels,P=P)

In [8]:
from pprint import pprint
print(t[3,4])
print(t[4,3])


[1.1577 3.4176 5.6932 7.9698]
[0.332 0.332 0.332 0.332]


In [9]:
def extract_min(Q,distance):
    minimo=float("inf") #highest python value
    nome=""
    for q in Q: #find the smallest
        if distance[q] <= minimo:
            nome=q
            minimo=distance[q]
    Q.remove(nome) #remove the correct node from the que
    return nome # tell me wich node is removed

def W(u,v,weight_matrix):
    for i in adj[u]:
        if i.end==v:
            return i.weight
        
def dijkstra(source,weight_matrix,l=0):
    distance=dict() #dict for distance from source
    parent=dict() #dict for parentnes of each node
    nodes=np.array([i for i in range(len(weight_matrix))]) #list of current node to iterate
    for node in nodes:
        distance[node]=float("inf") 
        parent[node]=None
    distance[source]=0
    S=[]
    Q={node for node in nodes} #set used ad que, coupled with distance dict
    while len(Q)>0:
        u=extract_min(Q,distance) #extract the min from Q,using distance dict
        S.append(u) #ultimated nodes
        for v,w in enumerate(weight_matrix[u]): #for each node v, outgoing from u, with weight w
            if distance[v] > distance[u] + w[l]: #relax phase
                distance[v]= distance[u] + w[l]
                parent[v]=u
        #print(f"{u = }")
        #print(f"{distance = }")
    return distance,parent #return distances from source and each node parent !
            
dist,parent = dijkstra(0,t)
pprint(dist)
pprint(parent)

{0: 0,
 1: 0.38,
 2: 1.4763000000000002,
 3: 2.5689,
 4: 3.3916,
 5: 3.6915999999999998,
 6: 4.0356,
 7: 5.315099999999999,
 8: 5.2126,
 9: 6.8701,
 10: 6.2101,
 11: 5.761,
 12: 3.2565999999999997,
 13: 3.1090999999999998,
 14: 4.5546,
 15: 6.0489999999999995,
 16: 6.0104999999999995,
 17: 2.3936,
 18: 1.9616,
 19: 1.1192,
 20: 2.1546000000000003}
{0: None,
 1: 0,
 2: 1,
 3: 2,
 4: 18,
 5: 4,
 6: 5,
 7: 13,
 8: 12,
 9: 10,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 12,
 15: 1,
 16: 14,
 17: 18,
 18: 0,
 19: 0,
 20: 19}


In [10]:
for i in t[:,:,0]:
    for j in i:
        print(f"\t{round(j,3)}",end=" ")
    print("\n")

	0.0 	0.38 	1.668 	3.389 	4.616 	5.555 	6.573 	8.238 	9.195 	9.47 	7.159 	6.709 	4.213 	3.624 	6.033 	6.711 	7.416 	2.495 	1.962 	1.119 	2.23 

	0.38 	0.0 	1.096 	2.315 	3.487 	4.474 	5.551 	7.112 	7.875 	8.143 	5.83 	5.381 	2.877 	2.729 	5.461 	5.669 	7.053 	3.121 	2.866 	1.551 	1.967 

	0.696 	0.36 	0.0 	1.093 	2.251 	3.265 	4.385 	5.896 	6.774 	7.549 	5.473 	4.738 	2.527 	2.372 	5.105 	5.312 	6.696 	2.794 	2.718 	0.824 	2.27 

	1.01 	0.674 	0.314 	0.0 	1.158 	2.157 	3.156 	4.8 	5.761 	6.535 	5.564 	3.723 	2.73 	3.162 	5.571 	5.669 	7.032 	1.488 	3.677 	1.138 	0.988 

	1.342 	1.006 	0.646 	0.332 	0.0 	0.3 	0.646 	3.621 	4.978 	6.549 	5.578 	3.737 	3.89 	4.318 	6.72 	6.83 	8.19 	1.82 	1.43 	1.47 	1.32 

	1.642 	1.306 	0.946 	0.632 	0.99 	0.0 	0.344 	2.62 	3.932 	5.654 	4.752 	4.318 	4.939 	5.361 	7.761 	6.508 	8.023 	2.122 	2.969 	1.772 	1.62 

	1.988 	1.65 	1.292 	0.978 	2.015 	0.938 	0.0 	1.433 	2.722 	4.598 	4.578 	4.723 	6.14 	6.553 	8.011 	6.333 	7.848 	2.466 	4.027 	2.116 	1.964

In [11]:
%%timeit
dist,parent = dijkstra(0,t)


291 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
def shortest_path_matrix(t_ij_mat):
    sp_matrix=np.empty_like(t_ij_mat)
    nodes=np.array([i for i in range(t_ij_mat.shape[0])])
    levels=np.array([i for i in range(t_ij_mat.shape[-1])])
    print(levels)
    for source in nodes:
        for l in levels:
            distance,_t=dijkstra(source,t_ij_mat,l)
            sp_matrix[source,:,l]=list(distance.values())
    return sp_matrix



In [13]:

sp = shortest_path_matrix(t)
for i in sp[:,:,0]:
    for j in i:
        print(f"\t{round(j,2)}",end=" ")
    print("\n")
    

[0 1 2 3]
	0.0 	0.38 	1.48 	2.57 	3.39 	3.69 	4.04 	5.32 	5.21 	6.87 	6.21 	5.76 	3.26 	3.11 	4.55 	6.05 	6.01 	2.39 	1.96 	1.12 	2.15 

	0.38 	0.0 	1.1 	2.19 	3.35 	3.65 	3.99 	4.94 	4.83 	6.49 	5.83 	5.38 	2.88 	2.73 	4.17 	5.67 	5.63 	2.77 	2.34 	1.5 	1.97 

	0.7 	0.36 	0.0 	1.09 	2.25 	2.55 	2.89 	4.33 	4.48 	6.13 	5.47 	4.74 	2.53 	2.37 	3.82 	5.31 	5.28 	2.58 	2.66 	0.82 	1.86 

	1.01 	0.67 	0.31 	0.0 	1.16 	1.46 	1.8 	3.23 	4.52 	6.22 	5.56 	3.72 	2.73 	2.69 	4.03 	5.63 	5.48 	1.49 	2.59 	1.14 	0.99 

	1.34 	1.01 	0.65 	0.33 	0.0 	0.3 	0.64 	2.08 	3.37 	5.24 	5.05 	3.74 	3.06 	3.02 	4.36 	5.76 	5.82 	1.82 	1.43 	1.47 	1.32 

	1.64 	1.31 	0.95 	0.63 	0.99 	0.0 	0.34 	1.78 	3.07 	4.94 	4.75 	4.32 	3.36 	3.32 	4.66 	5.46 	5.89 	2.12 	2.42 	1.77 	1.62 

	1.99 	1.65 	1.29 	0.98 	1.93 	0.94 	0.0 	1.43 	2.72 	4.6 	4.58 	4.7 	3.71 	3.66 	4.59 	5.28 	5.72 	2.47 	3.36 	2.12 	1.96 

	2.4 	2.06 	1.7 	1.39 	1.06 	0.75 	0.41 	0.0 	1.79 	3.67 	4.03 	4.56 	4.12 	3.7 	3.66 	4.74 	5.12 	2.87 	2.4

In [82]:
l = [1,2]
print(id(l))

140190043822656
